In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import namedtuple, deque

In [ ]:
def select_action(state, epsilon, action_size):
    if(epsilon > np.random.rand()):
        action = np.random.choice(action_size)
        return(action)
    else:
        state = torch.from_numpy(state).float().unsqueeze(0)
        with torch.no_grad():
            action_values = q_network(state)
        return np.argmax(action_values.numpy())

In [3]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed):
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [69]:
state_size = 4
action_size = 12
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_rewards_history = []
max_memory = 1000
running_reward = 0
episode_count = 0
timestep_count = 0
update_after_actions = 4

max_episodes = 500
max_steps_per_episode = 1000
last_n_reward = 100
epsilon = 0.05
penalty = 1.00
epsilon_decay = 0.5
target_update_every = 4
num = 21

action_grid = {
        'syn_opt_effort': ['extreme','high', 'medium', 'low', 'none'],
        'syn_map_effort': ['high','medium','low'],
        'syn_generic_effort': ['high', 'medium', 'low', 'none']
    }

In [100]:
def get_reward(state, state_next):
    area_diff = state_next[0] - state[0]
    time_diff = state_next[1] - state[1]
    reward = 0
    if area_diff > 0 and time_diff > 0:
        reward = 3
    elif area_diff == 0 and time_diff > 0:
        reward = 2
    elif area_diff < 0 and time_diff > 0:
        reward = 1
    elif area_diff > 0 and time_diff == 0:
        reward = 2
    elif area_diff > 0 and time_diff == 0:
        reward = 1
    elif area_diff == 0 and time_diff == 0:
        reward = -1
    elif area_diff == 0 and time_diff < 0:
        reward = -1
    elif area_diff < 0 and time_diff == 0:
        reward = -1
    elif area_diff < 0 and time_diff < 0:
        reward = -4
    
    return reward

In [71]:
seed = 10
q_network = QNetwork(state_size, action_size, seed)

In [72]:
seed = 10
target_q_network = QNetwork(state_size, action_size, seed)

In [73]:
state = np.array([1, 2, 3, 4])

In [75]:
a = select_action(state,epsilon,action_size)
if(a < 5):
    print("syn_opt_effort " + action_grid["syn_opt_effort"][a])
elif(a >= 5 and a <8):
    print("syn_map_effort " + action_grid["syn_map_effort"][a - 5])
elif(a >= 8 and a <12):
    print("syn_generic_effort " + action_grid["syn_global_effort"][a - 8])

syn_opt_effort none


In [120]:
class DQNAgent:
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Networks
        self.q_network = QNetwork(state_size, action_size, seed)
        self.target_q_network = QNetwork(state_size, action_size, seed)

        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=5e-4)

        # Experience replay
        self.memory = deque(maxlen=int(1e5))
        self.batch_size = 64
        self.gamma = 0.99

    def act(self, state, epsilon=0.0):
        if random.random() < epsilon:
            return random.choice(np.arange(self.action_size))
        else:
            state = torch.from_numpy(state).float().unsqueeze(0)
            with torch.no_grad():
                action_values = self.q_network(state)
            return np.argmax(action_values.numpy())

    def step(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

        if len(self.memory) > self.batch_size:
            experiences = random.sample(self.memory, k=self.batch_size)
            self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = zip(*experiences)

        states = torch.from_numpy(np.vstack(states)).float()
        actions = torch.from_numpy(np.vstack(actions)).long()
        rewards = torch.from_numpy(np.vstack(rewards)).float()
        next_states = torch.from_numpy(np.vstack(next_states)).float()
        dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float()

        Q_targets_next = self.target_q_network(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        Q_expected = self.q_network(states).gather(1, actions)

        loss = nn.functional.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        self.soft_update(self.q_network, self.target_q_network, 1e-3)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [121]:
agent = DQNAgent(state_size, action_size, seed=42)

In [122]:
class ENV:
    def __init__(self):
        self.state = np.array([100,200,200,100])
        self.prev_state = np.array([0,0,0,0])
        self.action_grid = action_grid = {
        'syn_opt_effort': ['extreme','high', 'medium', 'low', 'none'],
        'syn_map_effort': ['high','medium','low'],
        'syn_generic_effort': ['high', 'medium', 'low', 'none']
    }
        
    def step(self, action):
        self.prev_state = self.state
        self.state = np.random.rand(4)
        reward = get_reward(self.prev_state, self.state)
        print(reward)
        
        return self.state, reward
    
    def reset(self):
        self.state = np.array([100,200,200,100])
        return self.state
    

In [123]:
env = ENV()

In [ ]:
num_episodes = 1000
max_timesteps = 1000
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01

for episode in range(1, num_episodes + 1):
    state = env.reset()
    total_reward = 0

    for t in range(max_timesteps):
        action = agent.act(state, epsilon)
        next_state, reward = env.step(action)
        done = 0
        agent.step(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward


    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    print(total_reward)

-4
-4
1
-4
0
0
3
-4
1
0
1
-4
1
1
0
-4
3
-4
3
3
-4
3
1
0
3
3
0
-4
3
1
0
1
3
0
3
1
0
-4
1
0
1
1
0
-4
0
-4
0
1
0
3
-4
1
-4
3
1
0
-4
1
1
0
3
0
3
-4
3
3
-4
1
0
1
-4
1
3
0
1
-4
3
-4
1
-4
3
-4
0
3
-4
1
0
3
3
-4
-4
3
-4
3
-4
0
3
-4
1
0
3
-4
1
-4
0
3
1
3
-4
1
0
-4
3
3
0
1
-4
-4
0
1
3
1
-4
1
-4
0
1
1
3
0
-4
3
-4
0
1
1
3
0
-4
3
3
-4
1
3
1
0
0
-4
0
3
-4
0
0
1
-4
3
1
0
1
-4
0
1
1
1
3
0
3
-4
3
1
0
1
1
0
0
-4
3
-4
-4
1
0
-4
3
0
-4
0
3
1
0
3
0
1
3
-4
1
1
0
-4
3
-4
0
3
-4
0
-4
3
-4
-4
3
3
-4
3
-4
0
3
-4
-4
3
-4
0
3
-4
3
-4
-4
3
1
0
3
1
-4
1
0
3
-4
1
0
3
0
-4
3
-4
1
3
-4
-4
3
3
0
-4
3
1
3
-4
3
-4
-4
3
1
1
0
3
-4
3
-4
1
3
3
-4
3
3
-4
3
-4
0
0
1
0
3
-4
1
-4
-4
3
-4
-4
1
3
-4
0
1
-4
0
3
0
-4
1
-4
0
3
3
-4
3
0
0
-4
1
0
1
-4
1
1
0
3
1
1
3
-4
3
1
-4
3
1
3
0
3
-4
3
0
1
3
1
0
-4
3
0
0
-4
3
-4
0
1
-4
3
-4
1
0
3
0
1
3
0
1
-4
3
3
-4
1
1
0
1
0
-4
0
0
1
3
1
-4
3
3
1
3
-4
0
1
0
3
3
3
-4
-4
-4
1
0
0
1
0
-4
0
1
3
3
0
1
-4
3
1
0
1
3
3
-4
0
1
3
0
1
0
0
1
0
1
-4
0
3
1
0
1
0
-4
3
1
-4
0
1
-4
0
1
0
-4
0
3
1
0
3
-4
-4
0
1
3


-4
1
3
1
0
1
1
-4
0
3
-4
1
-4
0
1
3
0
1
1
-4
3
-4
0
1
-4
3
1
-4
3
0
1
-4
3
1
0
-4
3
3
1
-4
0
-4
3
-4
3
-4
3
1
0
3
0
1
3
-4
0
1
-4
-4
3
-4
1
0
0
3
3
-4
0
1
0
1
0
1
3
-4
1
0
1
-4
1
0
3
0
0
1
0
1
1
-4
3
3
1
3
-4
3
1
3
0
-4
1
3
-4
3
1
0
0
3
-4
3
1
3
-4
3
1
0
-4
1
1
3
-4
1
0
3
-4
1
3
-4
0
0
-4
3
1
0
1
3
0
1
-4
3
-4
-4
0
-4
0
3
1
0
3
1
0
0
3
3
-4
1
0
0
3
-4
3
3
0
-4
3
-4
3
1
0
1
-4
3
-4
3
1
3
0
-4
3
1
0
3
0
1
1
0
1
-4
3
1
-4
-4
3
-4
3
1
-4
3
0
1
1
3
-4
3
1
0
-4
-4
3
1
1
3
-4
3
0
3
-4
1
-4
3
-4
3
1
0
0
-4
1
0
1
0
3
-4
3
1
-4
0
0
1
-4
3
3
0
1
1
-4
3
1
1
1
0
0
3
0
1
-4
3
3
-4
3
-4
3
1
-4
3
-4
3
1
97
-4
3
0
1
-4
-4
3
1
-4
3
-4
3
0
-4
1
0
3
-4
0
1
1
0
-4
3
3
1
3
0
1
0
1
-4
0
1
0
3
-4
1
3
-4
0
3
0
-4
0
3
-4
1
0
0
-4
3
-4
3
-4
0
1
0
0
1
1
0
-4
3
0
1
3
-4
1
0
3
0
1
0
1
-4
3
-4
1
-4
0
1
0
3
-4
0
1
3
1
0
1
0
3
-4
1
3
0
-4
3
1
3
0
1
3
-4
3
1
0
3
-4
-4
1
3
-4
1
0
1
3
1
0
-4
0
-4
3
0
-4
1
-4
3
-4
0
3
1
-4
3
0
-4
0
3
-4
1
3
-4
3
-4
3
1
3
-4
1
0
1
3
-4
3
0
1
3
0
-4
-4
3
0
-4
0
1
-4
0
1
0
1
0
-4
3
3
1
-4
3


1
0
1
3
-4
3
1
-4
3
0
1
0
0
1
1
3
-4
0
1
0
3
1
0
3
-4
3
1
-4
1
-4
3
3
-4
0
1
3
-4
3
1
0
-4
0
1
0
1
0
-4
-4
3
3
-4
1
3
-4
3
0
1
3
0
-4
3
-4
3
1
-4
3
0
1
0
1
3
0
-4
3
1
-4
3
-4
-4
3
1
0
3
1
-4
3
1
3
-4
3
3
0
1
0
-4
-4
-4
3
0
1
0
3
-4
3
-4
0
1
3
0
1
-4
0
1
3
1
0
3
-4
3
1
3
-4
0
0
1
0
-4
3
-4
0
0
-4
3
-4
0
3
-4
1
0
-4
1
0
3
1
0
1
0
3
-4
3
1
0
1
3
0
1
0
1
-4
0
1
0
-4
3
0
1
0
1
3
-4
0
1
-4
1
0
0
-4
3
-4
0
1
3
1
0
-4
0
3
-4
3
0
-4
3
-4
3
1
3
0
1
1
0
-4
3
-4
3
1
-4
3
-4
3
1
0
1
1
-4
3
-4
3
-4
1
3
1
-4
3
0
1
1
3
0
3
-4
0
-4
1
3
-4
1
1
-4
3
-4
3
0
-4
1
0
-4
3
1
0
1
0
-4
-4
0
1
-4
-4
3
1
0
-4
1
-4
1
0
3
1
0
-4
1
3
-4
0
-4
3
3
0
-4
0
3
-4
-4
3
0
-4
3
1
0
1
-4
1
-4
1
3
0
1
0
3
-4
-4
1
3
0
0
-4
3
0
1
3
1
0
3
-4
3
-4
3
0
1
0
1
-4
3
-4
3
0
0
1
0
1
3
-4
0
1
0
1
3
0
0
-4
3
1
0
3
-4
1
3
-4
-4
3
1
-4
3
-4
3
3
0
1
0
3
-4
3
-4
3
-4
3
-4
0
-4
3
1
3
0
1
3
-4
-4
3
0
3
-4
3
-4
1
3
0
3
1
0
3
-4
0
1
3
0
1
3
-4
3
1
-4
3
0
3
0
-4
3
0
1
3
-4
3
-4
-4
-4
3
-4
3
0
1
0
1
3
0
-4
0
0
-4
1
-4
0
3
1
-4
3
0
1
0
1
0
1
-4
3
-4

0
1
0
-4
3
-4
1
-4
-4
3
-4
0
3
-4
3
1
0
3
3
-4
1
3
1
0
-4
-4
3
-4
1
3
-4
-4
1
0
0
3
1
0
1
-4
1
0
1
-4
0
-4
3
1
3
-4
3
0
-4
1
-4
0
-4
3
1
-4
0
0
1
0
3
3
-4
3
-4
0
1
1
0
1
1
0
1
1
-4
-4
3
0
1
0
-4
1
3
-4
3
1
0
-4
3
1
0
0
3
-4
3
1
0
1
0
1
3
0
1
-4
0
-4
-4
3
1
0
3
0
0
3
1
1
0
3
1
0
3
0
-4
1
0
1
3
-4
3
-4
3
-4
1
0
3
1
0
3
1
0
-4
3
1
-4
-4
0
1
0
-4
1
3
3
-4
-4
3
0
3
-4
1
-4
3
3
1
-4
3
-4
1
3
-4
1
0
-4
3
1
0
-4
3
-4
-4
3
-4
0
-4
3
1
1
0
1
-4
3
0
0
0
-4
3
3
-4
1
-4
3
3
1
3
-4
1
0
-4
1
-4
1
0
0
1
0
-4
0
-4
3
-4
0
3
1
3
-4
3
1
-4
3
1
0
-4
3
1
0
1
1
0
3
0
1
1
0
-4
1
3
1
1
0
1
0
1
1
0
-4
3
-4
3
-4
3
0
1
3
0
-4
3
-4
3
-4
3
0
3
3
-4
3
-4
3
0
0
1
3
-4
1
1
-4
3
1
0
-4
3
1
0
0
-4
1
3
0
1
1
3
-4
3
-4
1
0
1
3
-4
-4
3
1
0
-4
0
3
1
0
1
0
-4
0
1
-4
1
3
0
1
3
-4
3
-4
-4
3
0
1
0
-4
3
3
-4
3
1
0
1
3
1
0
-4
1
-4
0
1
1
0
0
1
0
1
0
1
-4
3
0
1
0
3
-4
0
3
1
0
1
0
-4
0
1
0
3
3
-4
1
3
1
0
-4
1
0
-4
0
1
-4
3
3
3
0
1
0
1
-4
3
1
0
1
0
-4
3
1
-4
0
1
3
3
0
1
3
-4
3
-4
1
0
1
0
-4
3
0
3
1
0
-4
0
3
0
1
3
-4
3
-4
0
-4
3
0
-4


3
-4
0
3
-4
3
-4
1
0
3
1
1
0
0
1
-4
3
1
0
3
0
-4
3
0
-4
1
3
-4
1
0
1
-4
3
-4
3
1
3
0
1
0
1
0
1
-4
3
-4
3
0
0
1
3
-4
3
1
0
1
0
0
-4
3
-4
0
1
0
0
3
1
-4
3
0
-4
3
1
3
0
0
0
1
-4
-4
0
-4
3
-4
0
-4
-4
1
3
3
0
0
1
-4
0
0
-4
3
0
1
0
-4
3
-4
0
3
3
-4
1
0
-4
1
3
-4
0
3
1
-4
0
0
1
1
0
1
3
0
-4
0
1
-4
3
0
-4
0
1
3
-4
1
0
1
0
1
-4
3
0
-4
3
-4
3
-4
3
0
-4
1
3
-4
1
0
0
1
3
-4
0
-4
-4
3
-4
1
-4
1
0
1
0
-4
3
0
1
1
0
3
-4
3
0
1
0
1
0
3
-4
-4
3
-4
0
1
0
1
3
0
-4
3
-4
3
-4
3
3
-4
-4
0
1
3
-4
0
1
0
1
0
3
-4
3
-4
0
1
0
-4
3
1
3
0
-4
0
3
0
-4
-4
1
3
1
0
-4
3
0
-4
3
1
0
1
0
1
-104
-4
3
-4
0
0
-4
1
3
-4
3
0
1
-4
0
1
-4
1
3
3
-4
-4
3
-4
0
1
1
0
3
-4
0
-4
3
1
3
-4
3
1
1
-4
0
1
3
-4
-4
1
3
-4
3
1
3
1
0
-4
-4
3
0
1
-4
0
3
-4
3
-4
0
3
-4
1
1
0
3
-4
1
0
3
0
1
-4
1
3
0
3
3
-4
3
-4
0
-4
1
0
3
1
3
0
-4
1
0
3
1
-4
0
3
-4
3
0
1
0
-4
1
1
0
-4
3
1
-4
-4
3
-4
3
-4
3
-4
3
3
-4
1
0
-4
1
3
-4
3
1
0
-4
3
1
1
0
-4
3
0
1
0
3
-4
-4
0
1
-4
0
-4
0
-4
3
-4
3
3
-4
3
1
0
1
-4
0
3
1
-4
3
-4
-4
0
3
0
-4
1
3
0
3
3
3
-4
3
1
0
3
0
1
0
1
3


...

-4
-4
3
3
0
1
3
-4
-4
-4
3
-4
1
1
0
1
0
1
3
1
0
-4
0
1
0
1
0
3
1
0
-4
3
-4
1
-4
3
1
0
-4
0
1
0
1
3
-4
1
1
0
3
-4
1
0
3
-4
3
-4
3
0
-4
1
3
-4
1
0
1
0
3
1
-4
3
-4
1
0
1
0
-4
0
1
0
0
0
1
1
-4
3
-4
1
3
-4
1
0
1
3
-4
0
1
1
3
-4
3
0
-4
3
1
3
0
1
3
-4
3
0
3
0
1
0
3
1
-4
0
1
-4
-4
0
1
-4
3
1
-4
3
1
0
3
0
1
1
0
3
-4
3
-4
3
-4
3
1
0
0
1
-4
-4
3
0
-4
1
3
-4
1
3
0
0
1
1
0
1
-4
3
1
0
3
-4
1
0
0
1
3
1
-4
0
1
0
1
1
0
1
3
1
0
1
0
-4
3
1
0
3
1
0
-4
1
3
0
3
-4
0
1
1
-4
3
1
-4
0
3
0
1
3
-4
1
0
3
-4
1
0
-4
3
-4
0
1
3
-4
3
0
1
0
1
3
0
1
1
0
0
1
-4
1
3
0
0
-4
1
0
1
0
0
1
1
3
1
0
1
0
0
1
3
1
46
-4
0
0
0
3
1
-4
1
-4
0
-4
0
1
0
3
0
-4
3
-4
1
0
3
0
1
0
3
-4
3
-4
3
3
-4
3
-4
-4
0
1
0
0
-4
1
3
-4
1
1
0
1
3
1
0
3
1
-4
0
1
0
1
-4
0
-4
3
1
-4
1
0
3
-4
3
-4
3
1
-4
-4
0
3
0
-4
1
3
1
-4
0
-4
3
-4
0
1
0
-4
1
1
0
-4
3
0
3
-4
-4
1
0
3
-4
3
-4
0
3
-4
0
3
3
-4
1
1
3
1
-4
3
-4
0
-4
3
3
-4
-4
-4
1
0
1
3
-4
3
3
-4
1
1
0
1
-4
0
1
3
-4
0
3
0
1
0
-4
1
3
-4
0
3
3
3
-4
1
-4
0
-4
0
1
0
1
3
1
0
0
0
1
1
0
-4
3
-4
3
3
1
-4
3
-4
0
1
0
0

0
-4
1
0
3
1
-4
0
3
1
-4
1
0
1
0
1
3
-4
1
-4
3
-4
3
1
0
1
-4
1
0
1
3
0
3
-4
0
3
0
0
-4
0
1
0
-4
3
1
1
3
-4
-4
3
-4
3
1
0
1
0
3
-4
0
-4
0
1
-4
-4
0
3
0
1
1
-4
0
3
0
-4
1
0
1
0
-4
3
3
-4
3
0
3
-4
3
1
0
1
1
0
1
3
-4
-4
0
3
0
-4
1
3
0
-4
1
1
0
-4
-4
3
1
0
3
-4
1
0
-4
1
0
0
3
3
1
0
-4
1
-4
0
3
1
0
1
0
1
1
0
-4
1
-4
0
1
-4
1
0
1
0
1
3
-4
0
1
-4
0
0
3
1
1
0
-4
3
0
1
1
0
3
-4
1
-4
-4
0
-4
1
0
1
0
3
1
-4
0
3
-4
1
0
1
-4
0
1
0
-4
3
-4
1
3
0
3
0
1
0
0
1
0
1
1
0
-4
1
-4
-4
3
-4
1
1
0
0
3
-4
1
-4
3
3
0
3
1
0
1
0
1
3
1
3
-4
0
-4
3
1
1
-4
3
0
1
0
1
0
3
-4
0
3
1
0
-4
3
-4
3
0
1
-4
3
-4
3
0
-4
3
-4
-4
3
0
1
-4
3
-4
3
-4
3
-4
1
1
-4
3
-4
3
0
-4
3
-4
0
1
-4
3
3
-4
-4
1
0
1
1
0
3
-4
-4
1
1
0
0
1
-4
0
-4
3
-4
3
0
0
3
0
1
1
0
3
-4
0
1
0
1
1
0
3
-4
3
0
3
1
-4
1
3
0
-4
1
0
3
-4
1
3
-4
0
0
1
-4
3
0
1
0
1
-4
3
1
0
-4
-4
1
0
0
-4
3
0
1
0
1
1
0
1
-4
3
0
3
1
0
1
0
1
-4
0
1
3
-4
0
3
1
0
-4
3
1
-4
-4
3
3
0
-4
-4
3
0
-4
0
1
1
0
1
-4
1
3
1
-4
3
1
0
1
0
3
0
-4
3
3
1
-4
1
3
1
0
1
0
1
1
-4
0
3
-4
1
0
1
1
0
1
1
0
-4
1
0
1

3
0
3
1
-4
3
0
0
1
-4
0
3
-4
3
-4
3
-4
0
3
1
0
1
-4
0
-4
3
0
-4
3
3
-4
1
3
0
1
0
1
3
0
-4
0
1
0
1
-4
-4
3
-4
1
3
0
0
1
-4
0
3
-4
3
0
1
1
-4
1
3
-4
3
-4
3
1
0
3
-4
3
3
-4
3
-4
0
1
1
0
3
-4
-4
-4
3
1
0
3
-4
1
-4
0
3
1
0
1
0
-4
3
-4
3
3
1
0
0
-4
3
0
0
3
1
0
1
-4
0
0
3
-4
-4
3
-4
0
-4
0
1
0
0
1
1
0
3
0
1
1
0
1
3
-4
-4
3
0
3
-4
0
1
3
1
-4
0
1
0
-4
1
-4
3
-4
0
-4
3
-4
0
-4
3
3
-4
-4
3
1
-4
3
-4
3
3
0
-4
3
1
0
3
3
-4
1
-4
1
3
0
-4
-4
1
0
3
0
1
3
3
-4
3
0
3
-4
-4
1
-4
3
0
0
-4
3
-4
1
0
-4
3
0
1
-4
3
0
-4
3
0
3
0
1
0
0
-4
3
0
3
-4
3
0
-4
1
3
3
1
0
0
1
0
1
0
1
3
1
-4
3
-4
3
0
1
0
0
1
1
-4
0
1
3
3
-4
0
3
0
-4
3
1
-4
3
1
1
0
-4
1
0
3
0
1
0
0
-4
3
-4
1
0
-4
3
3
0
-4
1
0
3
0
3
1
3
0
1
3
-4
0
1
-4
0
-4
0
1
0
-4
3
-4
-4
0
-4
0
3
-4
-4
1
-4
3
1
-4
3
-4
0
1
3
3
0
0
1
1
0
-4
1
3
1
-4
3
0
0
3
1
-4
3
-4
1
0
1
3
-4
3
0
0
1
3
-4
1
3
1
0
3
0
3
-4
0
1
0
1
0
3
-4
3
-4
0
-4
3
-4
1
0
1
1
-4
0
1
0
3
1
3
0
1
-4
1
3
3
1
-4
1
0
1
3
1
3
0
-4
3
0
-4
0
3
1
0
1
3
0
-4
3
1
3
0
1
-4
3
1
0
-4
1
3
-4
3
3
-4
1
-4
1
-4
3
-4
-4

1
-4
3
3
0
3
-4
3
3
0
-4
-4
3
-4
1
-4
3
-4
1
0
1
1
3
1
-4
3
1
3
0
1
3
0
-4
-4
1
0
1
0
0
1
-4
0
0
1
3
3
1
-4
0
1
0
1
0
-4
3
-4
1
3
-4
0
3
3
-4
3
1
0
1
1
0
0
-4
3
-4
3
1
0
1
-4
1
0
3
-4
3
1
-4
-4
3
0
1
0
1
3
1
0
3
-4
3
-4
0
-4
3
0
3
-4
-4
3
-4
0
-4
3
-4
-4
1
0
0
1
1
0
3
1
3
0
3
3
-4
1
0
1
3
1
0
-4
1
3
-4
3
-4
3
-4
-4
0
3
1
0
-4
3
1
-4
-4
0
3
-4
-4
3
1
3
-4
1
0
1
0
3
-4
0
1
0
3
0
1
0
0
1
1
0
1
0
-4
3
-4
3
-4
3
1
3
-4
0
3
3
3
-4
1
0
0
1
3
1
-4
3
-4
0
1
0
1
3
-4
3
0
1
0
-4
3
1
3
-4
3
1
0
1
0
-4
3
3
-4
1
0
-4
3
3
1
-4
0
1
3
0
0
1
-4
0
3
1
0
1
3
1
3
-4
0
1
0
-4
0
3
0
1
3
-4
3
1
3
1
0
-4
3
-4
0
-4
0
1
3
0
1
3
-4
1
0
3
1
0
0
1
0
1
0
-4
0
3
1
0
-4
1
0
-4
3
0
3
1
3
3
-4
3
-4
1
3
-4
1
-4
3
-4
3
0
1
-4
3
-4
3
0
1
3
0
-4
0
0
1
0
1
0
1
3
0
1
0
3
1
-4
1
94
-4
0
3
3
0
1
1
0
1
0
3
1
-4
3
-4
1
0
-4
1
0
1
-4
1
0
3
1
0
3
0
-4
1
0
0
-4
-4
3
1
0
-4
1
0
-4
1
0
3
-4
0
-4
1
0
1
0
1
3
3
-4
3
0
1
0
1
1
-4
1
0
-4
3
0
1
-4
1
3
0
1
1
0
-4
3
3
0
1
0
-4
3
0
1
0
0
-4
1
0
1
-4
3
1
1
-4
-4
-4
3
1
-4
3
-4
3
1
-4
0
3
3
-4


1
3
0
1
-4
0
-4
3
-4
1
0
1
-4
0
1
3
-4
0
1
1
1
0
3
0
1
3
0
1
0
0
1
3
1
0
3
1
3
0
-4
-4
0
1
0
-4
3
0
1
0
-4
3
-4
1
3
-4
1
1
3
-4
0
3
3
-4
1
0
0
1
0
1
3
0
1
-4
-4
1
3
-4
3
3
-4
0
-4
1
0
1
-4
0
3
1
-4
0
3
3
0
1
-4
-4
1
0
0
3
1
0
1
0
-4
3
-4
-4
3
0
1
3
1
0
3
0
1
0
1
-4
1
1
-4
3
0
3
-4
1
-4
1
3
-4
3
3
-4
0
3
-4
3
-4
1
3
-4
1
0
0
3
3
-4
-4
3
-4
3
-4
1
0
1
0
1
-4
3
0
3
1
-4
3
-4
3
-4
3
-4
0
1
-4
3
1
0
3
-4
0
1
3
-4
0
1
0
3
-4
1
-4
-4
0
1
0
1
0
0
-4
-4
3
-4
3
-4
3
3
-4
3
3
3
3
-4
1
-4
0
1
3
-4
3
3
-4
1
3
1
3
-4
0
-4
1
3
-4
1
3
-4
0
1
0
0
3
-4
-4
0
1
-4
3
-4
3
1
-4
0
3
-4
1
0
1
3
0
1
3
-4
0
1
-4
3
-4
-4
3
-4
1
0
1
0
0
1
0
1
-4
1
3
1
3
-4
0
-4
-4
3
1
1
0
0
3
0
1
0
1
0
3
1
0
1
0
0
1
0
1
1
0
3
-4
0
1
1
0
3
3
-4
1
0
3
3
-4
1
-4
3
0
3
3
-4
1
-4
3
0
-4
3
-4
3
-4
0
1
3
1
0
-4
1
0
-4
3
3
-4
3
0
3
-4
-4
0
1
1
-4
1
3
1
-4
-4
1
3
0
-4
3
1
0
3
-4
1
0
3
-4
3
1
3
0
0
3
-4
-4
3
3
-4
-4
3
0
1
-4
3
0
1
0
1
0
0
-4
1
3
-4
3
1
0
-4
0
1
-4
3
-4
0
1
0
0
1
0
1
1
0
1
-4
3
-4
1
3
3
-4
-4
3
0
3
-4
1
0
0
1
-4
3
0
-4
3
-4

In [ ]:
reward